In [1]:
import time
from datetime import datetime
import datetime
import cv2
import base64
import numpy as np
import json
import requests
import pandas as pd
import re
import dask.dataframe as dd
import sqlite3 as sqlite
import re
import time

url_cebrace_salvar = "http://localhost:22700/api/ocorrencia/salvar"
url_cebrace_update = "http://localhost:22700/api/ocorrencia/update_foto"
headers_empresa = {'content-type': 'application/json'}
#headers_cebrace = {"content-type": "application/x-www-form-urlencoded"}


In [10]:
conn = sqlite.connect('safety_camera1.db')
cursor = conn.cursor()

In [11]:
dataframe =  'bancodeImagens1.csv'

df =  pd.read_csv(dataframe,
                  sep = ',', 
                  error_bad_lines = False,
                 encoding='utf-8',
                 )
#df =  dd.read_csv(dataframe)

In [12]:
df['status'] = 10
df[df['status'] == 10]
df.head()

,idimage,idcamera,acquisition_time,update_time,status,json_returned,eztask_return,raw_image,processed_image,anomaly_group,accuracy_check
0,4,4,2019-06-25 08:02:53,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2054, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,NaN,0.0
1,11,4,2019-06-25 08:02:56,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2055, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,NaN,0.0
2,18,4,2019-06-25 08:02:59,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1187, ""class"":...","{""id"": 2056, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,NaN,0.0
3,25,4,2019-06-25 08:03:02,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2057, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,NaN,0.0
4,32,4,2019-06-25 08:03:05,2019-06-28 10:53:54,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2058, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,NaN,0.0


In [13]:
df['acquisition_time'] =  pd.to_datetime(df['acquisition_time'],format="%Y-%m-%d %H:%M:%S")
type(df['acquisition_time'])

pandas.core.series.Series

In [14]:
df.to_sql("images", conn,if_exists='append', index=False)

In [7]:
#cursor.execute("""SELECT * FROM images Where status = 10;""")

#cursor.execute("""SELECT *,(SELECT COUNT()+1 FROM (SELECT DISTINCT idcamera FROM images AS t WHERE status = 10)) AS Rank FROM images;""")

#resultadoConsulta =  cursor.fetchall()
#resultadoConsulta[0]

In [8]:
for image in resultadoConsulta:  
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    stime= time.time()
    anomalies = json.loads(image[5])
    anomaly_txt= ""
    for item in anomalies:
        if "safety" in item:
            anomaly_txt += "\nAndando fora da faixa de segurança"
        elif "no helmet" in anomalies[item]["class"]:
            anomaly_txt += "\nFalta de capacete"
        elif "no vest" in anomalies[item]["class"]:
            anomaly_txt += "\nFalta de colete"
    ##### trecho para enviar imagem marcada
    jpg_as_text = image[8]
    jpg_as_text = jpg_as_text.encode("UTF-8")
    jpg_as_text = base64.b64encode(jpg_as_text).decode('utf-8')
    #####
    data_cebrace = json.dumps({ "anexo": jpg_as_text , "dataInicial": image[2].strftime("%Y-%m-%d %H:%M:%S"), "dataFinal": now, "camera": ("Unidade " + "JCR" + " - Local " + "colocar local" ), "anomalia": anomaly_txt })
    response_cebrace = requests.post(url_cebrace, data = data_cebrace, headers = headers_cebrace)
 

NameError: name 'resultadoConsulta' is not defined

In [9]:
df = pd.read_sql_query("SELECT * FROM images Where status = 10", conn, index_col =None)

In [10]:
df['acquisition_time'] = pd.to_datetime(df.acquisition_time)


In [11]:
df['acquisition_time'] = df['acquisition_time'].dt.floor('120s')


In [12]:
df['acquisition_time'] = df['acquisition_time'].factorize()[0]
df1_grouped = df.groupby('acquisition_time')

In [13]:
df.head()

,idimage,idcamera,acquisition_time,update_time,status,json_returned,eztask_return,raw_image,processed_image,anomaly_group,accuracy_check
0,4,4,0,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2054, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
1,11,4,0,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2055, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
2,18,4,0,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1187, ""class"":...","{""id"": 2056, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
3,25,4,0,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2057, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
4,32,4,0,2019-06-28 10:53:54,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2058, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0


In [2]:
df['acquisition_time'] = df['acquisition_time'].factorize()[0]
testtet =  df.groupby(level=[0]).apply(lambda g:  g.iloc[-1,0])

NameError: name 'df' is not defined

In [3]:
#for index, image in df.iterrows(): 
#for index, image in df.iloc[:2].iterrows():
i= 0 
for index , image in df1_grouped:
    i+=1
    
    #print(index)
    #print(type(image))
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    stime= time.time()
    anomalies = json.loads(index)
    anomaly_txt= ""
    for item in anomalies:
        if "safety" in item:
            anomaly_txt += "\nAndando fora da faixa de segurança"
        elif "no helmet" in anomalies[item]["class"]:
            anomaly_txt += "\nFalta de capacete"
        elif "no vest" in anomalies[item]["class"]:
            anomaly_txt += "\nFalta de colete"
    ##### trecho para enviar imagem marcada
    jpg_as_text = image[5]
    jpg_as_text = jpg_as_text.encode("UTF-8")
    jpg_as_text = base64.b64encode(jpg_as_text).decode('utf-8')
    #####
    data_cebrace = json.dumps({"dataInicial": "2019-09-27T09:39:40", "dataFinal": "2019-09-27T09:52:36", "cameraID": 7, "anomaliaID": 107, "anexoID": None, "camera": "5", "anomalia": "Falta de capacete", "anexo": None, "dataCriacao": "2019-09-27T09:52:36.487", "dataModificacao": None, "ativo": True})
    #data_cebrace = json.dumps({ "anexo": jpg_as_text , "dataInicial": image[2], "dataFinal": now, "camera": ("Unidade " + "JCR" + " - Local " + "colocar local" ), "anomalia": anomaly_txt })
    response_cebrace_salvar = requests.post(url_cebrace_salvar, data = data_cebrace, headers = headers_cebrace)
    if response_cebrace_salvar.status_code == 400:
        print("Erro ao enviar as images")
    elif response_cebrace.status_code == 200:
        obj = json.loads(response_cebrace.text)
        #print(obj['id'])
        for update in image.iterrows():
            #print(update)
            jsonupdate =  json.dumps({"AtividadeID": 2845,"Anexo" : jpg_as_text})
            response_cebrace_update = requests.post(url_cebrace_update, data = jsonupdate, headers = headers_cebrace)
            print(response_cebrace_update.text)
        
        


NameError: name 'df1_grouped' is not defined